# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 15 2022 9:01AM,244132,12,HC-81622,HealthCaps LLC,Released
1,Aug 15 2022 8:58AM,244131,10,BPS20021,Beach Patient Assistance,Released
2,Aug 15 2022 8:58AM,244131,10,BPS20021,Beach Patient Assistance,Released
3,Aug 15 2022 8:58AM,244131,10,BPS20020,Beach Patient Assistance,Released
4,Aug 15 2022 8:58AM,244131,10,BPS20022,Beach Patient Assistance,Released
5,Aug 15 2022 8:52AM,244130,16,SAR00000359,Sartorius Stedim North America,Released
6,Aug 15 2022 8:49AM,244129,10,PRF-35043,Bio-PRF,Released
7,Aug 15 2022 8:37AM,244127,10,8395115,Beach Products Inc,Released
8,Aug 15 2022 8:37AM,244127,10,8404224,Beach Products Inc,Released
9,Aug 15 2022 8:37AM,244127,10,8404225,Beach Products Inc,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
13,244127,Released,13
14,244129,Released,1
15,244130,Released,1
16,244131,Released,4
17,244132,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
244127,NaN,13.0
244129,NaN,1.0
244130,NaN,1.0
244131,NaN,4.0
244132,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
244107,0.0,10.0
244108,6.0,5.0
244109,0.0,27.0
244110,14.0,1.0
244111,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
244107,0,10
244108,6,5
244109,0,27
244110,14,1
244111,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,244107,0,10
1,244108,6,5
2,244109,0,27
3,244110,14,1
4,244111,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,244107,,10
1,244108,6,5
2,244109,,27
3,244110,14,1
4,244111,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance
5,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America
6,Aug 15 2022 8:49AM,244129,10,Bio-PRF
7,Aug 15 2022 8:37AM,244127,10,Beach Products Inc
20,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.
26,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc."
36,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation
47,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation
74,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,,1
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,,4
2,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,,1
3,Aug 15 2022 8:49AM,244129,10,Bio-PRF,,1
4,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,,13
5,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,,6
6,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",,10
7,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,6,5
8,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,,27
9,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,14,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,1,
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,4,
2,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,1,
3,Aug 15 2022 8:49AM,244129,10,Bio-PRF,1,
4,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,
5,Aug 15 2022 8:34AM,244122,20,Alumier Labs Inc.,6,
6,Aug 15 2022 8:30AM,244107,10,"Methapharm, Inc.",10,
7,Aug 15 2022 8:30AM,244108,10,ISDIN Corporation,5,6
8,Aug 15 2022 8:30AM,244109,10,ISDIN Corporation,27,
9,Aug 15 2022 8:30AM,244110,10,ISDIN Corporation,1,14


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,1,
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,4,
2,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,1,
3,Aug 15 2022 8:49AM,244129,10,Bio-PRF,1,
4,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,1,NaN
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,4,NaN
2,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,1,NaN
3,Aug 15 2022 8:49AM,244129,10,Bio-PRF,1,NaN
4,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Aug 15 2022 9:01AM,244132,12,HealthCaps LLC,1,0.0
1,Aug 15 2022 8:58AM,244131,10,Beach Patient Assistance,4,0.0
2,Aug 15 2022 8:52AM,244130,16,Sartorius Stedim North America,1,0.0
3,Aug 15 2022 8:49AM,244129,10,Bio-PRF,1,0.0
4,Aug 15 2022 8:37AM,244127,10,Beach Products Inc,13,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2441176,73,20.0
12,732369,37,0.0
16,244130,1,0.0
20,488242,8,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2441176,73,20.0
1,12,732369,37,0.0
2,16,244130,1,0.0
3,20,488242,8,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,73,20.0
1,12,37,0.0
2,16,1,0.0
3,20,8,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,73.0
1,12,Released,37.0
2,16,Released,1.0
3,20,Released,8.0
4,10,Executing,20.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,20
Status,,,,
Executing,20.0,0.0,0.0,0.0
Released,73.0,37.0,1.0,8.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,20
0,Executing,20.0,0.0,0.0,0.0
1,Released,73.0,37.0,1.0,8.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,20
0,Executing,20.0,0.0,0.0,0.0
1,Released,73.0,37.0,1.0,8.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()